In [1]:
import pandas as pd
import pyam
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import aneris

# Start of Processing the Results

In [4]:
image_path = "../../../data/SOD/model_results/raw/Consolidated_IMAGE_GEO7_R3_results_12022025_0956.xlsx"
aim_path = "../../../data/SOD/model_results/raw/Consolidated_AIM_GEO7_240621.xlsx"
globio_path = "../../../data/SOD/model_results/processed/GLOBIO_results_processed.xlsx"

In [5]:
variables_chpt_19_from_iiasa_image = [
    "Emissions|CH4",
    "Emissions|CO2",
    "Emissions|N2O",
    "Emissions|CO2|AFOLU",
    "Emissions|CO2|Energy",
    # 'Emissions|F-Gases',
    "Emissions|Kyoto Gases",
    "Land Cover|Built-up Area",
    "Land Cover|Cropland",
    "Land Cover|Forest",
    "Land Cover|Other Natural Land",
    "Land Cover|Pasture",
    "Population",
    "Primary Energy",
    "Primary Energy|Biomass",
    "Primary Energy|Biomass|Traditional",
    "Primary Energy|Coal",
    "Primary Energy|Gas",
    "Primary Energy|Non-Biomass Renewables",
    "Primary Energy|Nuclear",
    "Primary Energy|Oil",
    "Secondary Energy|Electricity",
    "Secondary Energy|Electricity|Biomass",
    "Secondary Energy|Electricity|Coal",
    "Secondary Energy|Electricity|Gas",
    "Secondary Energy|Electricity|Non-Biomass Renewables",
    "Secondary Energy|Electricity|Nuclear",
    "Secondary Energy|Electricity|Oil",
    "Final Energy",
    "Final Energy|Bunkers",
    "Final Energy|Industry",
    "Final Energy|Other Sector",
    "Final Energy|Residential and Commercial",
    "Final Energy|Transportation",
    "Yield|Cereal",
    "Food Demand",
    "Food Demand|Crops",
    "Food Demand|Livestock",
    "Final Energy|Electricity",
    "Agricultural Production|Non-Energy|Crops",
    "Agricultural Production|Crops|Non-Energy",
]

variables_chpt_19_from_iiasa_aim = [
    "Emissions|CH4",
    "Emissions|CO2",
    "Emissions|N2O",
    # 'Emissions|F-Gases',
    "Emissions|Kyoto Gases",
    "Land Cover|Built-up Area",
    "Land Cover|Cropland",
    "Land Cover|Forest",
    "Land Cover|Other Natural Land",
    "Land Cover|Pasture",
    "Population",
    "Primary Energy",
    "Primary Energy|Non-Biomass Renewables",
    "Primary Energy|Biomass|Traditional",
    "Primary Energy|Biomass",
    "Secondary Energy|Electricity",
    "Final Energy",
    "Yield|Cereal",
    "Food Demand",
    "Food Demand|Crops",
    "Food Demand|Livestock",
    "Final Energy|Electricity",
    "Agricultural Production|Non-Energy|Crops",
]

regions_chpt_19 = [
    "IMAGE 3.3|Canada",
    "IMAGE 3.3|Japan",
    "IMAGE 3.3|Mexico",
    "IMAGE 3.3|South Africa",
    "IMAGE 3.3|Turkey",
    "IMAGE 3.3|India",
    "IMAGE 3.3|Rest of Southern Africa",
    "IMAGE 3.3|Brazil",
    "IMAGE 3.3|China Region",
    "IMAGE 3.3|Eastern Africa",
    "IMAGE 3.3|Indonesia Region",
    "IMAGE 3.3|Northern Africa",
    "IMAGE 3.3|Rest of South Asia",
    "IMAGE 3.3|Russia Region",
    "IMAGE 3.3|USA",
    "IMAGE 3.3|Western Africa",
    "IMAGE 3.3|Western Europe",
    "IMAGE 3.3|Oceania",
]

image_short_to_long_dict = {
    "BRA": "IMAGE 3.3|Brazil",
    "CAN": "IMAGE 3.3|Canada",
    "CEU": "IMAGE 3.3|Central Europe",
    "CHN": "IMAGE 3.3|China Region",
    "EAF": "IMAGE 3.3|Eastern Africa",
    "INDIA": "IMAGE 3.3|India",
    "INDO": "IMAGE 3.3|Indonesia Region",
    "JAP": "IMAGE 3.3|Japan",
    "KOR": "IMAGE 3.3|Korea Region",
    "ME": "IMAGE 3.3|Middle East",
    "MEX": "IMAGE 3.3|Mexico",
    "NAF": "IMAGE 3.3|Northern Africa",
    "OCE": "IMAGE 3.3|Oceania",
    "RCAM": "IMAGE 3.3|Central America",
    "RSAF": "IMAGE 3.3|Rest of Southern Africa",
    "RSAM": "IMAGE 3.3|Rest of South America",
    "RSAS": "IMAGE 3.3|Rest of South Asia",
    "RUS": "IMAGE 3.3|Russia Region",
    "SAF": "IMAGE 3.3|South Africa",
    "SEAS": "IMAGE 3.3|Southeastern Asia",
    "STAN": "IMAGE 3.3|Central Asia",
    "TUR": "IMAGE 3.3|Turkey",
    "UKR": "IMAGE 3.3|Ukraine Region",
    "USA": "IMAGE 3.3|USA",
    "WAF": "IMAGE 3.3|Western Africa",
    "WEU": "IMAGE 3.3|Western Europe",
}

df_image_all = pyam.IamDataFrame(
    image_path
)
# rename regions to extened names
df_image_all.rename(region=image_short_to_long_dict, inplace=True)
# rename variable so it matches AIM
df_image_all.rename(
    variable={
        "Agricultural Production|Crops|Non-Energy": "Agricultural Production|Non-Energy|Crops",
    },
    inplace=True,
)
# Filter on transpaths
df_image = df_image_all.filter(
    variable=variables_chpt_19_from_iiasa_image,
    scenario=["TECH-TP-v2", "LIFE-TP-v2"],
    region=regions_chpt_19,
    model="IMAGE*",
)
# filter on REF, to be used later for harmonization
df_image_hist = df_image_all.filter(
    variable=variables_chpt_19_from_iiasa_aim,
    scenario=["REF-v2"],
    region=regions_chpt_19,
    model="IMAGE*",
)

regions_chpt_19_aim = [
    "CAN",
    "JPN",
    "USA",
    "XE25",
    "XOC",
    "BRA",
    "TUR",
    "CHN",
    "CIS",
    "IND",
    "XNF",
    "XAF",  # Also containes south africa
    "XSA",
]

# Read in AIM data from local copy (if data still on db, it can also be read from there)
df_aim = pyam.IamDataFrame(
    aim_path
)
df_aim = df_aim.filter(
    variable=variables_chpt_19_from_iiasa_aim,
    scenario=["REF-v2"],
    region=regions_chpt_19_aim,
)

globio_vars = [
    "Terrestrial Biodiversity|MSA",
    "Terrestrial Biodiversity|MSA|Plants",
    "Terrestrial Biodiversity|MSA|Vertebrates",
    "Area",
]

df_globio = pyam.IamDataFrame(
    globio_path
)
df_globio.rename(region=image_short_to_long_dict, inplace=True)
df_globio = df_globio.filter(
    variable=globio_vars,
    scenario=["REF-v2", "TECH-TP-v2", "LIFE-TP-v2"],
    region=regions_chpt_19,
)
# Use only IMAGe results for the GDP baseline
df_gdp = df_image_all.filter(
    variable=[
        "GDP|PPP",
    ],
    scenario=["REF-v2", "TECH-TP-v2", "LIFE-TP-v2"],
    region=regions_chpt_19,
    model="IMAGE*",
)

df_image = pyam.concat([df_image, df_gdp])


# Create aggregated land cover variables
# ------------------------------------------------------------------------------------------------------------
# Land Cover|Forest and Other Natural Land
df_image.aggregate(
    "Land Cover|Forest and Other Natural Land",
    components=["Land Cover|Forest", "Land Cover|Other Natural Land"],
    append=True,
)
df_image = df_image.filter(
    variable=["Land Cover|Forest", "Land Cover|Other Natural Land"], keep=False
)

df_image = df_image.rename(variable={"Land Cover|Built-up Area": "Land Cover|Other"})

# Land Cover|Forest and Other Natural Land
df_image_hist.aggregate(
    "Land Cover|Forest and Other Natural Land",
    components=["Land Cover|Forest", "Land Cover|Other Natural Land"],
    append=True,
)
df_image_hist = df_image_hist.filter(
    variable=["Land Cover|Forest", "Land Cover|Other Natural Land"], keep=False
)
df_image_hist = df_image_hist.rename(
    variable={"Land Cover|Built-up Area": "Land Cover|Other"}
)

# Land Cover|Forest and Other Natural Land
df_aim.aggregate(
    "Land Cover|Forest and Other Natural Land",
    components=["Land Cover|Forest", "Land Cover|Other Natural Land"],
    append=True,
)

df_aim = df_aim.filter(
    variable=["Land Cover|Forest", "Land Cover|Other Natural Land"], keep=False
)
df_aim = df_aim.rename(variable={"Land Cover|Built-up Area": "Land Cover|Other"})
# ----------------------------------------------------------------------------------------------------------


df_image.timeseries()

[INFO] 10:37:15 - pyam.core: Reading file ..\..\..\data\SOD\model_results\raw\Consolidated_IMAGE_GEO7_R3_results_12022025_0956.xlsx


[INFO] 10:37:28 - pyam.core: Reading file ..\..\..\data\SOD\model_results\raw\Consolidated_AIM_GEO7_240621.xlsx
[INFO] 10:37:35 - pyam.core: Reading file ..\..\..\data\SOD\model_results\processed\GLOBIO_results_processed.xlsx


2005  \
model     scenario   region                   variable                                           unit                           
IMAGE 3.3 LIFE-TP-v2 IMAGE 3.3|Brazil         Agricultural Production|Non-Energy|Crops           million t DM/yr   256.077422   
                                              Emissions|CH4                                      Mt CH4/yr          20.884529   
                                              Emissions|CO2                                      Mt CO2/yr        1959.565226   
                                              Emissions|CO2|AFOLU                                Mt CO2/yr        1606.474921   
                                              Emissions|CO2|Energy                               Mt CO2/yr         332.711950   
...                                                                                                                       ...   
          TECH-TP-v2 IMAGE 3.3|Western Europe Secondary Energy|Electricity|Gas                   EJ/yr               2.272719   
                                              Secondary Energy|Electricity|Non-Biomass Renewa... EJ/yr               1.863086   
                                              Secondary Energy|Electricity|Nuclear               EJ/yr               3.327559   
                                              Secondary Energy|Electricity|Oil                   EJ/yr               0.462214   
                                              Yield|Cereal                                       t DM/ha/yr          4.921615   

                                                                                                                         2010  \
model     scenario   region                   variable                                           unit                           
IMAGE 3.3 LIFE-TP-v2 IMAGE 3.3|Brazil         Agricultural Production|Non-Energy|Crops           million t DM/yr   350.607750   
                                              Emissions|CH4                                      Mt CH4/yr          22.332270   
                                              Emissions|CO2                                      Mt CO2/yr        2180.096293   
                                              Emissions|CO2|AFOLU                                Mt CO2/yr        1760.023195   
                                              Emissions|CO2|Energy                               Mt CO2/yr         388.272124   
...                                                                                                                       ...   
          TECH-TP-v2 IMAGE 3.3|Western Europe Secondary Energy|Electricity|Gas                   EJ/yr               2.603184   
                                              Secondary Energy|Electricity|Non-Biomass Renewa... EJ/yr               2.398619   
                                              Secondary Energy|Electricity|Nuclear               EJ/yr               3.067513   
                                              Secondary Energy|Electricity|Oil                   EJ/yr               0.328678   
                                              Yield|Cereal                                       t DM/ha/yr          5.187173   

                                                                                                                        2015  \
model     scenario   region                   variable                                           unit                          
IMAGE 3.3 LIFE-TP-v2 IMAGE 3.3|Brazil         Agricultural Production|Non-Energy|Crops           million t DM/yr  411.691531   
                                              Emissions|CH4                                      Mt CH4/yr         17.956930   
                                              Emissions|CO2                                      Mt CO2/yr        914.278936   
                                              Emissions|CO2|AFOLU                                Mt CO2/yr        398.508375 

In [6]:
weighted_variables = [
    'Food Demand',
    'Food Demand|Crops',
    'Food Demand|Livestock',
    'Yield|Cereal'
]
abs_variables = [value for value in df_image.variable if value not in weighted_variables]

# aggregate IMAGE and GLOBIO

In [7]:
def compute_3_groups_image(df, variables, weight=None):
    df_high_income = df.aggregate_region(variables,
                              region='High-Income',
                              subregions=[
                                'IMAGE 3.3|Canada',
                                'IMAGE 3.3|Japan',
                                'IMAGE 3.3|USA',
                                'IMAGE 3.3|Western Europe',
                                'IMAGE 3.3|Oceania'
                              ],
                              weight=weight
                              )

    df_middle_income = df.aggregate_region(variables,
                              region='Middle-Income',
                              subregions=[
                                'IMAGE 3.3|Mexico',
                                'IMAGE 3.3|South Africa',
                                'IMAGE 3.3|Turkey',
                                'IMAGE 3.3|Brazil',
                                'IMAGE 3.3|China Region',
                                'IMAGE 3.3|Indonesia Region',
                                'IMAGE 3.3|Russia Region',
                              ],
                              weight=weight
                              )

    df_low_income = df.aggregate_region(variables,
                              region='Low-Income',
                              subregions=[
                                'IMAGE 3.3|India',
                                'IMAGE 3.3|Rest of Southern Africa',
                                'IMAGE 3.3|Eastern Africa',
                                'IMAGE 3.3|Northern Africa',
                                'IMAGE 3.3|Rest of South Asia',
                                'IMAGE 3.3|Western Africa',
                              ],
                              weight=weight
                              )

    df_3_groups = pyam.concat([df_high_income, df_middle_income, df_low_income])
    return df_3_groups


def compute_6_groups_image(df, variables, weight=None):
    df_high_low = df.aggregate_region(variables,
                              region='High-Income, Low Density',
                              subregions=[
                                'IMAGE 3.3|Canada',
                                'IMAGE 3.3|Oceania',
                                'IMAGE 3.3|USA',
                              ],
                              weight=weight
                              )

    df_middle_low = df.aggregate_region(variables,
                              region='Middle-Income, Low Density',
                              subregions=[
                                'IMAGE 3.3|Mexico',
                                'IMAGE 3.3|South Africa',
                                'IMAGE 3.3|Brazil',
                                'IMAGE 3.3|Russia Region',
                              ],
                              weight=weight
                              )

    df_low_low = df.aggregate_region(variables,
                              region='Low-Income, Low Density',
                              subregions=[
                                'IMAGE 3.3|Rest of Southern Africa',
                              ],
                              weight=weight
                              )

    df_high_high = df.aggregate_region(variables,
                              region='High-Income, High Density',
                              subregions=[
                                'IMAGE 3.3|Japan',
                                'IMAGE 3.3|Western Europe',
                              ],
                              weight=weight
                              )

    df_middle_high = df.aggregate_region(variables,
                              region='Middle-Income, High Density',
                              subregions=[
                                'IMAGE 3.3|Turkey',
                                'IMAGE 3.3|China Region',
                              ],
                              weight=weight
                              )

    df_low_high = df.aggregate_region(variables,
                              region='Low-Income, High Density',
                              subregions=[
                                'IMAGE 3.3|India',
                                'IMAGE 3.3|Rest of South Asia',
                              ],
                              weight=weight
                              )

    df_6_groups = pyam.concat([df_high_low,
                              df_middle_low,
                              df_low_low,
                              df_high_high,
                              df_middle_high,
                              df_low_high])
    return df_6_groups


def compute_3_groups_aim(df, variables, weight=None):
    df_high_income = df.aggregate_region(variables,
                              region='High-Income',
                              subregions=[
                                "CAN",
                                "JPN",
                                "USA",
                                "XE25",
                                "XOC",
                              ],
                              weight=weight
                              )

    df_middle_income = df.aggregate_region(variables,
                              region='Middle-Income',
                              subregions=[
                                "BRA",
                                "TUR",
                                "CHN",
                                "CIS",
                              ],
                              
                              weight=weight
                              )

    df_low_income = df.aggregate_region(variables,
                              region='Low-Income',
                              subregions=[
                                "IND",
                                "XNF",
                                "XAF",  # Also containes south africa
                                "XSA",
                              ],
                              
                              weight=weight
                              )

    df_3_groups = pyam.concat([df_high_income, df_middle_income, df_low_income])
    return df_3_groups


def compute_6_groups_aim(df, variables, weight=None):
    df_high_low = df.aggregate_region(variables,
                              region='High-Income, Low Density',
                              subregions=[
                                "CAN",
                                "USA",
                                "XOC",
                              ],
                              weight=weight
                              )

    df_middle_low = df.aggregate_region(variables,
                              region='Middle-Income, Low Density',
                              subregions=[
                                "BRA",
                                "CIS",
                              ],
                              weight=weight
                              )

    # df_low_low = df.aggregate_region(variables,
    #                           region='Low-Income, Low Density',
    #                           subregions=[
                                
    #                           ],
    #                           weight=weight
    #                           )

    df_high_high = df.aggregate_region(variables,
                              region='High-Income, High Density',
                              subregions=[
                                "JPN",
                                "XE25",
                              ],
                              weight=weight
                              )

    df_middle_high = df.aggregate_region(variables,
                              region='Middle-Income, High Density',
                              subregions=[
                                "TUR",
                                "CHN",
                              ],
                              weight=weight
                              )

    df_low_high = df.aggregate_region(variables,
                              region='Low-Income, High Density',
                              subregions=[
                                "IND",
                                "XSA",
                              ],
                              weight=weight
                              )

    df_6_groups = pyam.concat([df_high_low,
                              df_middle_low,
                              # df_low_low,
                              df_high_high,
                              df_middle_high,
                              df_low_high])
    return df_6_groups


df_6_groups_image_abs = compute_6_groups_image(df_image, abs_variables)
df_3_groups_image_abs = compute_3_groups_image(df_image, abs_variables)

df_6_groups_image_food = compute_6_groups_image(df_image, ['Food Demand','Food Demand|Crops','Food Demand|Livestock',], 'Population')
df_3_groups_image_food = compute_3_groups_image(df_image, ['Food Demand','Food Demand|Crops','Food Demand|Livestock',], 'Population')
df_6_groups_image_yield = compute_6_groups_image(df_image, ['Yield|Cereal',], 'Agricultural Production|Non-Energy|Crops')
df_3_groups_image_yield = compute_3_groups_image(df_image, ['Yield|Cereal',], 'Agricultural Production|Non-Energy|Crops')

df_6_groups_image = pyam.concat([df_6_groups_image_abs, df_6_groups_image_food, df_6_groups_image_yield])
df_3_groups_image = pyam.concat([df_3_groups_image_abs, df_3_groups_image_food, df_3_groups_image_yield])

df_6_groups_globio = compute_6_groups_image(df_globio, globio_vars, weight='Area')
df_3_groups_globio = compute_3_groups_image(df_globio, globio_vars, weight='Area')

df_6_groups_image_hist_abs = compute_6_groups_image(df_image_hist, abs_variables)
df_3_groups_image_hist_abs = compute_3_groups_image(df_image_hist, abs_variables)
df_6_groups_image_hist_food = compute_6_groups_image(df_image_hist, ['Food Demand','Food Demand|Crops','Food Demand|Livestock',], 'Population')
df_3_groups_image_hist_food = compute_3_groups_image(df_image_hist, ['Food Demand','Food Demand|Crops','Food Demand|Livestock',], 'Population')
df_6_groups_image_hist_yield = compute_6_groups_image(df_image_hist, ['Yield|Cereal',], 'Agricultural Production|Non-Energy|Crops')
df_3_groups_image_hist_yield = compute_3_groups_image(df_image_hist, ['Yield|Cereal',], 'Agricultural Production|Non-Energy|Crops')

df_6_groups_image_hist = pyam.concat([df_6_groups_image_hist_abs, df_6_groups_image_hist_food, df_6_groups_image_hist_yield])
df_3_groups_image_hist = pyam.concat([df_3_groups_image_hist_abs, df_3_groups_image_hist_food, df_3_groups_image_hist_yield])

df_3_groups_aim_abs = compute_3_groups_aim(df_aim, abs_variables)
df_6_groups_aim_abs = compute_6_groups_aim(df_aim, abs_variables)
df_6_groups_aim_food = compute_6_groups_aim(df_aim, ['Food Demand','Food Demand|Crops','Food Demand|Livestock'], 'Population')
df_3_groups_aim_food = compute_3_groups_aim(df_aim, ['Food Demand','Food Demand|Crops','Food Demand|Livestock'], 'Population')
df_6_groups_aim_yield = compute_6_groups_aim(df_aim, ['Yield|Cereal'], 'Agricultural Production|Non-Energy|Crops')
df_3_groups_aim_yield = compute_3_groups_aim(df_aim, ['Yield|Cereal'], 'Agricultural Production|Non-Energy|Crops')

df_6_groups_aim = pyam.concat([df_6_groups_aim_abs, df_6_groups_aim_food, df_6_groups_aim_yield])
df_3_groups_aim = pyam.concat([df_3_groups_aim_abs, df_3_groups_aim_food, df_3_groups_aim_yield])

# df_3_groups_aim.timeseries()

# Harmonize

In [8]:
# Create a dataframe that contains the method of harmonization per variable
# Only create methods for variables present in AIM
methods = df_6_groups_aim.timeseries().reset_index(level='model', drop=True)
# The offset methods ensure internal regional consistency better than the ratio methods
methods['method'] = 'reduce_offset_2080' # 'reduce_ratio_2080' 'constant_ratio' 'constant_offset' 'reduce_offset_2080'
methods_series = methods['method']

# These variables need a different method, manually checked for best performance
land_cover_vars = ['Land Cover|Other',
                    'Land Cover|Cropland',
                    'Land Cover|Forest and Other Natural Land',
                    'Land Cover|Pasture',
                    ]


methods_series.loc[:,:,land_cover_vars,:] = 'constant_offset'
methods_series

C:\Users\maycocks\AppData\Local\Temp\2\ipykernel_11856\3351199307.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  methods_series.loc[:,:,land_cover_vars,:] = 'constant_offset'


scenario  region                      variable                                  unit           
REF-v2    High-Income, High Density   Agricultural Production|Non-Energy|Crops  million t DM/yr    reduce_offset_2080
                                      Emissions|CH4                             Mt CH4/yr          reduce_offset_2080
                                      Emissions|CO2                             Mt CO2/yr          reduce_offset_2080
                                      Emissions|Kyoto Gases                     Mt CO2-equiv/yr    reduce_offset_2080
                                      Emissions|N2O                             kt N2O/yr          reduce_offset_2080
                                                                                                          ...        
          Middle-Income, Low Density  Primary Energy|Biomass                    EJ/yr              reduce_offset_2080
                                      Primary Energy|Biomass|Traditional      

In [9]:
# AIM will be harmonized to IMAGE REF historic values
def harmonize_df(df_aim, df_image_hist):

    # # Create a dataframe that contains the method of harmonization per variable
    # # Only create methods for variables present in AIM
    methods = df_aim.timeseries().reset_index(level='model', drop=True)
    # The offset methods ensure internal regional consistency better than the ratio methods
    methods['method'] = 'reduce_offset_2080' # 'reduce_ratio_2080' 'constant_ratio' 'constant_offset' 'reduce_offset_2080'
    methods_series = methods['method']

    # These variables need a different method, manually checked for best performance
    land_cover_vars = ['Land Cover|Other',
                        'Land Cover|Cropland',
                        'Land Cover|Forest and Other Natural Land',
                        'Land Cover|Pasture',
                        ]

    methods_series.loc[:,:,land_cover_vars,:] = 'constant_offset'
    if len(df_aim.region) == 6:
        methods_series.loc[:,['Low-Income, Low Density'],:,:] = 'constant_offset'

    hist_to_harmonize = df_image_hist.timeseries()
    df_aim_to_harmonize = df_aim.timeseries()

    # Remove the model level from index and column names should be strings
    df_aim_to_harmonize = df_aim_to_harmonize.reset_index(level='model', drop=True)
    df_aim_to_harmonize.columns = df_aim_to_harmonize.columns.astype(str)
    hist_to_harmonize = hist_to_harmonize.reset_index(level='model', drop=True)
    hist_to_harmonize.columns = hist_to_harmonize.columns.astype(str)

    # Harmonize 
    h = aneris.harmonize.Harmonizer(df_aim_to_harmonize, hist_to_harmonize, config={'harmonize_year': '2020'})
    harmonized = h.harmonize(methods_series) # methods_copy
    # Add all necessary index columns
    new_level = 'AIM_Harmonized'
    df_index = harmonized.index.to_frame()  # Convert the MultiIndex to a DataFrame
    df_index['model'] = new_level  # Add the new level
    df_index = df_index[['model', 'scenario', 'region', 'variable', 'unit']]
    new_index = pd.MultiIndex.from_frame(df_index)  # Create a new MultiIndex with the added level
    harmonized.index = new_index

    df_harmonized = pyam.IamDataFrame(harmonized)
    return df_harmonized

df_6_groups_aim_harmonized = harmonize_df(df_6_groups_aim, df_6_groups_image_hist)
df_6_groups_aim_harmonized = df_6_groups_aim_harmonized.filter(region='Low-Income, Low Density', keep=False)
df_3_groups_aim_harmonized = harmonize_df(df_3_groups_aim, df_3_groups_image_hist)


C:\Users\maycocks\AppData\Local\Temp\2\ipykernel_11856\187036843.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  methods_series.loc[:,:,land_cover_vars,:] = 'constant_offset'
REF-v2    Low-Income, Low Density  Agricultural Production|Non-Energy|Crops  million t DM/yr    (REF-v2, Low-Income, Low Density, Agricultural...
                                   Emissions|CH4                             Mt CH4/yr          (REF-v2, Low-Income, Low Density, Emissions|CH...
                                   Emissions|CO2                             Mt CO2/yr          (REF-v2, Low-Income, Low Density, Emissions|CO...
                                   Emissions|Kyoto Gases                     Mt CO2-equiv/yr    (REF-v2, Low-Income, Low Density, Emissions|Ky...
                                   Emissions|N2O 

In [10]:
# Concat all models
df_6_groups = pyam.concat([df_6_groups_image, df_6_groups_globio, df_6_groups_aim_harmonized])
df_3_groups = pyam.concat([df_3_groups_image, df_3_groups_globio, df_3_groups_aim_harmonized])
df_6_groups = df_6_groups.filter(year=[2010, 2020, 2025, 2030, 2035, 2040, 2045, 2050])
df_3_groups = df_3_groups.filter(year=[2010, 2020, 2025, 2030, 2035, 2040, 2045, 2050])

In [11]:
# Create per Capita variables
per_capita_vars = [
    'Emissions|Kyoto Gases|Per Capita',
    'Final Energy|Per Capita',
    'Primary Energy|Per Capita',
]

variables_chpt_19 =  df_6_groups_image.variable + per_capita_vars

def compute_per_capita_variables(df):
    df.divide('Primary Energy', "Population", "Primary Energy|Per Capita", append=True)
    df.divide('Final Energy', "Population", "Final Energy|Per Capita", append=True)
    df.convert_unit('GWh/yr', 'kWh/yr', inplace=True)
    df.divide(
            "Emissions|Kyoto Gases", "Population", "Emissions|Kyoto Gases|Per Capita", ignore_units="t CO2-equiv/yr", append=True)
    # df.divide("Water Withdrawal", "Population", "Water Withdrawal|Per Capita (temp)", ignore_units=True, append=True)
    # df.multiply("Water Withdrawal|Per Capita (temp)", 1000, "Water Withdrawal|Per Capita", ignore_units='m3/yr', append=True)
    df.convert_unit("EJ / a / million", "kWh/yr", inplace=True)
    
compute_per_capita_variables(df_6_groups)
compute_per_capita_variables(df_3_groups)
# compute_per_capita_variables(df_image)

In [13]:
to_share_vars_topics = {
    'Agricultural Demand': 'Land',
    'Agricultural Demand|Crops': 'Land',
    'Agricultural Demand|Energy': 'Land',
    'Agricultural Demand|Energy|Crops': 'Land',
    'Agricultural Demand|Non-Energy|Crops|Feed': 'Land',
    'Agricultural Demand|Non-Energy|Crops|Food': 'Land',
    'Agricultural Demand|Non-Energy|Crops|Other': 'Land',
    'Agricultural Demand|Non-Energy|Livestock': 'Land',
    'Agricultural Production': 'Land',
    'Agricultural Production|Crops': 'Land',
    'Agricultural Production|Energy|Crops': 'Land',
    'Agricultural Production|Non-Energy|Crops': 'Land',
    'Agricultural Production|Non-Energy|Livestock': 'Land',
    'Carbon Sequestration|CCS': 'Emissions',
    'Carbon Sequestration|CCS|Biomass': 'Emissions',
    'Carbon Sequestration|CCS|Fossil': 'Emissions',
    'Carbon Stocks': 'Land',
    'Carbon Stocks|Soil': 'Land',
    'Carbon Stocks|Vegetation': 'Land',
    'Emissions|CH4': 'Emissions',
    'Emissions|CH4|AFOLU': 'Emissions',
    'Emissions|CH4|Energy': 'Emissions',
    'Emissions|CO2': 'Emissions',
    'Emissions|CO2|AFOLU': 'Emissions',
    'Emissions|CO2|Energy': 'Emissions',
    'Emissions|Kyoto Gases': 'Emissions',
    'Emissions|N2O': 'Emissions',
    'Emissions|N2O|AFOLU': 'Emissions',
    'Emissions|N2O|Energy': 'Emissions',
    'Food Demand|Crops (Share)': 'Food',
    'GDP|PPP': 'Macro_Economy',
    'GDP|PPP|Per Capita': 'Macro_Economy',
    'Land Cover|Cropland': 'Land',
    'Land Cover|Forest and Other Natural Land': 'Land',
    'Land Cover|Other': 'Land',
    'Land Cover|Pasture': 'Land',
    'Population': 'Macro_Economy',
    'Population|Urban (Share)': 'Macro_Economy',
    'Primary Energy': 'Energy',
    'Primary Energy|Biomass': 'Energy',
    'Primary Energy|Biomass|w/ CCS': 'Energy',
    'Primary Energy|Biomass|w/o CCS': 'Energy',
    'Primary Energy|Coal': 'Energy',
    'Primary Energy|Coal|w/ CCS': 'Energy',
    'Primary Energy|Coal|w/o CCS': 'Energy',
    'Primary Energy|Fossil': 'Energy',
    'Primary Energy|Fossil|w/ CCS': 'Energy',
    'Primary Energy|Fossil|w/o CCS': 'Energy',
    'Primary Energy|Gas': 'Energy',
    'Primary Energy|Gas|w/ CCS': 'Energy',
    'Primary Energy|Gas|w/o CCS': 'Energy',
    'Primary Energy|Non-Biomass Renewables': 'Energy',
    'Primary Energy|Non-Biomass Renewables (Share)': 'Energy',
    'Primary Energy|Oil': 'Energy',
    'Primary Energy|Oil|w/ CCS': 'Energy',
    'Primary Energy|Oil|w/o CCS': 'Energy',
    'Primary Energy|Other': 'Energy',
    # 'Water Withdrawal': 'Water',
    'Agricultural Demand|Non-Energy|Crops': 'Land',
    'Agricultural Production|Non-Energy|Crops': 'Land',
    'Final Energy': 'Energy',
    'Final Energy|Electricity': 'Energy',
    'Final Energy|Industry': 'Energy',
    'Final Energy|Other Sector': 'Energy',
    'Final Energy|Residential and Commercial': 'Energy',
    'Final Energy|Transportation': 'Energy',
    'Final Energy|Bunkers': 'Energy',
    'Primary Energy|Solar': 'Energy',
    'Primary Energy|Wind': 'Energy',
    'Primary Energy|Hydro': 'Energy',
    'Primary Energy|Nuclear': 'Energy',
    'Water Consumption': 'Water',
    'Population|Clean Cooking Access': 'Socio_Economics',
    'Population|Electricity Access': 'Socio_Economics',
    'Population|Relying on Solid Fuels': 'Socio_Economics',
    'Temperature|Global Mean': 'Climate',
    'Primary Energy|Biomass|Traditional': 'Energy',
    'Carbon stocks': 'Land',
    'Carbon stocks|Soil': 'Land',
    'Carbon stocks|Vegetation': 'Land',
    'Food Waste': 'Food',
    'Food Waste|Consumption': 'Food',
    'Food Waste|Per capita': 'Food',
    'Food Waste|Production': 'Food',
    'Food Demand': 'Food',
    'Food Demand|Crops': 'Food',
    'Food Demand|Livestock': 'Food',
    'Yield|Cereal':'Land',
    'Emissions|Kyoto Gases|Per Capita':'Emissions',
    'Final Energy|Per Capita':'Energy',
    'Primary Energy|Per Capita':'Energy',
    'Secondary Energy|Electricity':'Energy',
    'Secondary Energy|Electricity|Biomass':'Energy',
    'Secondary Energy|Electricity|Coal':'Energy',
    'Secondary Energy|Electricity|Gas':'Energy',
    'Secondary Energy|Electricity|Non-Biomass Renewables':'Energy',
    'Secondary Energy|Electricity|Nuclear':'Energy',
    'Secondary Energy|Electricity|Oil':'Energy',
    'Area':"Biodiversity",
    'Terrestrial Biodiversity|MSA':"Biodiversity",
    'Terrestrial Biodiversity|MSA|Plants':"Biodiversity",
    'Terrestrial Biodiversity|MSA|Vertebrates':"Biodiversity",

}


In [14]:
# Chapter 21 also wanted data on income group level
chpt_21_vars = [
    'Yield|Cereal',
    'Emissions|CO2',
    'Emissions|Kyoto Gases',
    'Food Demand',
    'Food Demand|Crops',
    'Food Demand|Livestock',
    'Primary Energy',
    'Primary Energy|Non-Biomass Renewables',
    'Primary Energy|Biomass|Traditional',
    'Primary Energy|Biomass',
    'Final Energy',
    'Final Energy|Electricity',
    'GDP|PPP',
    'Population',
]

df_3_groups_chpt_21 = df_3_groups.filter(variable=chpt_21_vars)
df_6_groups_chpt_21 = df_6_groups.filter(variable=chpt_21_vars)

# Remove the variables that were for chapter 21 but not for chapter 19
df_3_groups = df_3_groups.filter(variable=[
    'Yield|Cereal',
    'Food Demand',
    'Food Demand|Crops',
    'Food Demand|Livestock',
    'Final Energy|Electricity',
    'Agricultural Production|Non-Energy|Crops'
], keep=False)

df_6_groups = df_6_groups.filter(variable=[
    'Yield|Cereal',
    'Food Demand',
    'Food Demand|Crops',
    'Food Demand|Livestock',
    'Final Energy|Electricity',
    'Agricultural Production|Non-Energy|Crops'
], keep=False)

# Only using non biomass renewables for chapter 21 data sharing
df_3_groups = df_3_groups.filter(variable=['Primary Energy|Non-Biomass Renewables', 
                                           'Primary Energy|Biomass|Traditional',
                                           'Primary Energy|Biomass'],scenario='REF-v2', keep=False)
df_6_groups = df_6_groups.filter(variable=['Primary Energy|Non-Biomass Renewables', 
                                           'Primary Energy|Biomass|Traditional',
                                           'Primary Energy|Biomass'],scenario='REF-v2', keep=False)

In [15]:
# Function to round each value to the order of magnitude of 1% of the value
def round_to_0_1_percent(value):
    if value == 0:
        return 0
    # Determine the number of significant digits to round to
    magnitude = np.floor(np.log10(abs(value))) - 2
    rounding_factor = 10 ** magnitude
    return np.round(value / rounding_factor) * rounding_factor

In [16]:
pandas_df_21 = pyam.concat([df_3_groups_chpt_21, df_6_groups_chpt_21], ignore_meta_conflict=True).timeseries().reset_index()
ratio_mask = pandas_df_21['unit'] == '%'
pandas_df_21.loc[ratio_mask, pandas_df_21.columns[5:]] = round(pandas_df_21.loc[ratio_mask, pandas_df_21.columns[5:]])

pandas_df_21.loc[:, pandas_df_21.columns[5:]] = pandas_df_21.loc[:, pandas_df_21.columns[5:]].applymap(round_to_0_1_percent)
pandas_df_21['model'] = pandas_df_21['variable'].map(to_share_vars_topics).fillna(pandas_df_21['model'])

pandas_df_21.scenario = pandas_df_21.scenario.replace({
    'REF-v2':'REF', 
    'TECH-TP-v2':'TECH-TP', 
    'LIFE-TP-v2':'LIFE-TP'})

file_path = '../../../data/SOD/model_results/to_share/model_results_to_share_Chapter_21_Income.xlsx'
# to_share = pyam.IamDataFrame(pandas_df_21)
pandas_df_21.to_excel(file_path, sheet_name="data", index=False)

C:\Users\maycocks\AppData\Local\Temp\2\ipykernel_11856\2441545439.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pandas_df_21.loc[:, pandas_df_21.columns[5:]] = pandas_df_21.loc[:, pandas_df_21.columns[5:]].applymap(round_to_0_1_percent)


In [17]:
pandas_df = pyam.concat([df_3_groups, df_6_groups], ignore_meta_conflict=True).timeseries().reset_index()
ratio_mask = pandas_df['unit'] == '%'
pandas_df.loc[ratio_mask, pandas_df.columns[5:]] = round(pandas_df.loc[ratio_mask, pandas_df.columns[5:]])

pandas_df.loc[:, pandas_df.columns[5:]] = pandas_df.loc[:, pandas_df.columns[5:]].applymap(round_to_0_1_percent)
pandas_df['model'] = pandas_df['variable'].map(to_share_vars_topics).fillna(pandas_df['model'])

pandas_df.scenario = pandas_df.scenario.replace({
    'REF-v2':'REF', 
    'TECH-TP-v2':'TECH-TP', 
    'LIFE-TP-v2':'LIFE-TP'})

file_path = '../../../data/SOD/model_results/to_share/Chapter_19_to_share.xlsx'
# to_share = pyam.IamDataFrame(pandas_df)
pandas_df.to_excel(file_path, sheet_name="data", index=False)

C:\Users\maycocks\AppData\Local\Temp\2\ipykernel_11856\3127856541.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pandas_df.loc[:, pandas_df.columns[5:]] = pandas_df.loc[:, pandas_df.columns[5:]].applymap(round_to_0_1_percent)
